In [ ]:
!pip install torch -U
!pip install transformers
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.3 MB/s eta 0:00:00


In [ ]:
!pip install emoji
!pip install soynlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 6.9 MB/s eta 0:00:00


In [ ]:
import transformers
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from transformers import EarlyStoppingCallback


import accelerate
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

### GPU 설정하기,

device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
print("device : " , device)

device :  cuda:0


### 필요한 토크나이저

In [ ]:
Model_name = 'beomi/KcELECTRA-base-v2022'
tokenizer = AutoTokenizer.from_pretrained(Model_name)

### 데이터 준비

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/감성분석/Data/Sentiment_0815_Analysis_data.csv",index_col = 0)

In [ ]:
data.index = range(len(data))

In [ ]:
import re
import emoji
from soynlp.normalizer import repeat_normalize
emojis = ''.join(emoji.EMOJI_DATA.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
def clean(x):
    x = pattern.sub(' ', str(x))
    x = emoji.replace_emoji(x, replace='') #emoji 삭제
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

#1.
import pandas as pd
#sentence_csv = pd.read_csv('ASD.csv',encoding='CP949',index_col=0)
#KAKAO_xlsx = pd.read_excel('cl_여가생활_2.xlsx', index_col=0)
#sentence_df = pd.DataFrame(KAKAO_xlsx)
#sentence_df.columns = ['sentence']
#raw_sen = sentence_df['sentence']
sentence = data['document']
#2.
clean_sen = []
for i in range(len(sentence)):
    temp = clean(sentence[i])
    clean_sen.append(temp)

#3.
def before_yok_bokone(x):
    #숫자 제거
    x = re.sub(r'[0-9]+', '', x)
    #특수문자 제거(자음,모음 하나씩 있는 것 살림)
    x = re.sub(r'[^\uAC00-\uD7A3ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㅏㅑㅓㅕㅗㅛㅜㅠㅡㅣ0-9a-zA-Z\s]', '', x)
    x = re.sub(r'(?<![ㄱ-ㅎㅏ-ㅣ])[ㄱ-ㅎㅏ-ㅣ](?![ㄱ-ㅎㅏ-ㅣ])', '', x)
    #공백이 두개 이상이면 하나만 남김
    x = re.sub(r'\s+', ' ', x)
    return x

#4.
before_yok = []
for i in range(len(clean_sen)):
    temp = before_yok_bokone(clean_sen[i])
    before_yok.append(temp)

#6.
def clean_chosung_english(x):
      #초성, 영어 제거
    x = re.sub(r'[^\uAC00-\uD7A3\s]', '', x)
    #문장 맨 마지막에 있는 공백 제거
    x = x.rstrip()
    return x

#7.
#after_yok은 욕 복원된 문장
final_clean_sen = []
for i in range(len(before_yok)):
    temp = clean_chosung_english(before_yok[i])
    final_clean_sen.append(temp)

#8.
#왼쪽 오른쪽 공백 제거

for i in range(len(final_clean_sen)):
    final_clean_sen[i] = final_clean_sen[i].strip()

### 시리즈 변환
data['document'] = pd.Series(final_clean_sen)

### 10개 이하 or 80개 이상 문장 없애기
data = data[(data['document'].str.len() > 10) & (data['document'].str.len() < 80)]

### 중복된 행 제거
data = data.drop_duplicates(subset='document', keep='first')

data.index = range(len(data))
#9.
#final_clean_sen_df = pd.DataFrame(final_clean_sen)
#final_clean_sen_df.columns=['sentence']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['document'], data['label'], test_size=0.2, shuffle = True)

In [ ]:
X_train

223985                                        재밌는데왜 흥행실패했을까
318162                                        딱히 뭐 나한테는 별로다
253280                                   멋진 영화다 아주 좋아 배우 분들
32073         다른제품은 다 괜찮은데 스워치 연결 커넥터는 전원이 들어 왔다 안들어 왔다 하네요
162480    컬러감이랑 대박이에요 암막 완벽 아침이되도 모름니다 역시 커튼은 밝은색이 최고에요 ...
                                ...                        
203101    아팬텀이랑 크리스틴이랑    부른거 계속 다시보게 됨  시각적으로도 화려하고노래 듣...
159806    따뜻한 그레이 소재 까슬거리지 않고 색상 예쁩니다 목선이 많이 파이지 않아서 좋습니...
89216                       배달이오다 깨진건지 약간깨져 왓는데 귀찬아서 그냥 씁니다
332995                              진짜 꼭해보세요 시간이 금방가고 진짜 갓겜
241643                                 주성치가 이런 실수를주성치의 최고졸작
Name: document, Length: 343432, dtype: object

In [ ]:
tokenized_train_sentence = tokenizer(
    list(X_train),
    return_tensors = 'pt',
    max_length = 128,
    padding = True,
    truncation = True,
    add_special_tokens = True)


tokenized_test_sentence = tokenizer(
    list(X_test),
    return_tensors = 'pt',
    max_length = 128,
    padding = True,
    truncation = True,
    add_special_tokens = True)

In [ ]:
class CurseDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

In [ ]:
train_label = y_train.values
test_label = y_test.values

train_dataset = CurseDataset(tokenized_train_sentence, train_label)
test_dataset = CurseDataset(tokenized_test_sentence, test_label)

## 모델 학습

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(Model_name, num_labels = 2)
model.to(device)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
training = TrainingArguments(
    output_dir='./',
    num_train_epochs = 40,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    logging_dir='./logs',
    logging_steps=500,
    save_total_limit=2,
    save_strategy = 'steps',
    evaluation_strategy="steps",  # 이를 "steps" 또는 "epoch"으로 설정하여 정기적으로 평가를 수행
    eval_steps=500,  # 평가를 얼마나 자주 수행할지 (500 스텝마다 평가)
    load_best_model_at_end=True,  # 학습이 끝날 때 최고의 모델을 로드
    metric_for_best_model="f1",  # 모델의 성능을 비교하기 위한 메트릭 설정 (여기서는 f1 점수)
    greater_is_better=True,  # f1 점수가 클수록 좋으므로 True로 설정
      # 개선이 없을 때 얼마나 많은 평가를 기다릴 것인지
)



def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision , recall, f1, _ = precision_recall_fscore_support(labels, preds, average = 'binary')
  acc = accuracy_score(labels, preds)
  return {
      'accuracy ' : acc,
      'f1': f1,
      'precision' : precision,
      'recall' : recall
  }



In [ ]:
trainer = Trainer(
    model=model,
    args=training,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 평가 데이터셋 지정
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-11-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.241200,0.352598,0.879232,0.871405,0.877116,0.865768
1000,0.184900,0.454528,0.878557,0.874396,0.855264,0.894404
1500,0.172800,0.365192,0.880583,0.875160,0.864934,0.885630
2000,0.176000,0.377818,0.883285,0.875235,0.884474,0.866187
2500,0.173200,0.381124,0.884846,0.877876,0.880040,0.875724
3000,0.161600,0.387668,0.875377,0.870325,0.856254,0.884867
3500,0.288800,0.303545,0.887478,0.880715,0.882535,0.878903
4000,0.287200,0.287007,0.887036,0.880154,0.882652,0.877671
4500,0.290100,0.281014,0.887769,0.880896,0.883670,0.878139
5000,0.286100,0.287225,0.884171,0.878460,0.871357,0.885680


<ipython-input-11-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-d3da3f7a2d94>:7: UserWarning: To copy construct from a 

TrainOutput(global_step=12000, training_loss=0.24968936665852864, metrics={'train_runtime': 11467.5495, 'train_samples_per_second': 1197.926, 'train_steps_per_second': 37.438, 'total_flos': 1.420710898962816e+16, 'train_loss': 0.24968936665852864, 'epoch': 1.12})

## 모델 평가

In [ ]:
trainer.evaluate(eval_dataset = test_dataset)

<ipython-input-11-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.2627009153366089,
 'eval_accuracy ': 0.8947693311126381,
 'eval_f1': 0.8902120420438665,
 'eval_precision': 0.8780803528622111,
 'eval_recall': 0.902683654106804,
 'eval_runtime': 275.3074,
 'eval_samples_per_second': 311.866,
 'eval_steps_per_second': 4.875,
 'epoch': 1.12}

## 예측

In [ ]:
def predict(sent):
  model.eval()

  tokenized_sent = tokenizer(
      sent,
      return_tensors = 'pt',
      truncation = True,
      add_special_tokens = True,
      max_length = 128
  )

  tokenized_sent.to(device)

  with torch.no_grad():
    outputs = model(
        input_ids = tokenized_sent['input_ids'],
        attention_mask = tokenized_sent['attention_mask'],
        token_type_ids = tokenized_sent['token_type_ids']
    )

  logits = outputs[0]
  logits = logits.detach().cpu()
  print(logits)
  result = logits.argmax(-1)

  if result == 0:
    result = '언어감수성이 낮습니다'
  elif result == 1:
    result = '언어감수성이 높습니다'
  return result

In [ ]:
# 모델 저장 경로 설정
save_path = "/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/감성분석/Result/8 22 결과"

# 모델과 토크나이저 저장
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


('/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/감성분석/Result/8 22 결과/tokenizer_config.json',
 '/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/감성분석/Result/8 22 결과/special_tokens_map.json',
 '/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/감성분석/Result/8 22 결과/vocab.txt',
 '/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/감성분석/Result/8 22 결과/added_tokens.json',
 '/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/감성분석/Result/8 22 결과/tokenizer.json')